In [4]:
%matplotlib inline
import matplotlib as mp
import pandas as pd
import numpy as np
import sklearn as sk
import sklearn.model_selection as skmp
import sklearn.ensemble as en
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
from sklearn.metrics import roc_curve

#warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [42]:
SD = pd.read_csv('Desktop/PFE.csv') #SD = stockData


In [40]:
SD = SD.assign(Buy = lambda x: (x.Close.shift(-5)>x.Close).astype('int'))


In [7]:
SD = SD.assign(MA = lambda x: (x['Close'].rolling(window = 30, min_periods = 9).mean()))
sum = 0
for i in range(30):
    sum += SD.Close[i]
    SD.MA[i] = sum/(i+1)

/Users/moisbourla/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
SD = SD.assign(upSwing = lambda x: (x.Close.shift(-100)>x.MA).astype('int'))

In [9]:
cols = [c for c in SD.columns if c != 'Buy']
x = SD[cols]
y =  SD.Buy
SD

,Open,High,Low,Close,Adj Close,Volume,Buy,MA,upSwing
0,31.990000,32.080002,31.530001,31.990000,26.756863,38775000,0,31.990000,0
1,31.549999,31.990000,31.540001,31.889999,26.673218,27196600,1,31.939999,0
2,31.740000,32.000000,31.719999,31.990000,26.756863,25023400,1,31.956666,0
3,31.870001,32.240002,31.830000,32.230000,26.957598,22753300,1,32.025000,0
4,32.119999,32.410000,31.850000,32.110001,26.857235,27517600,1,32.042000,0
5,31.520000,32.139999,31.520000,31.980000,26.748499,23791900,1,32.031667,0
6,32.340000,32.740002,32.340000,32.689999,27.342354,26325100,0,32.125714,0
7,32.650002,32.950001,32.599998,32.750000,27.392536,22451400,0,32.203750,0
8,32.799999,32.959999,32.410000,32.459999,27.149975,20520900,0,32.232222,0
9,32.509998,32.660000,32.180000,32.430000,27.124886,17610800,0,32.252000,0


In [13]:

x_train,x_test,y_train,y_test = sk.model_selection.train_test_split(x, y, test_size=0.2)
rf = en.RandomForestClassifier()
lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
rf.fit(x_train, y_train_encoded)
y_test_encoded = lab_enc.fit_transform(y_test)
rf.score(x_test, y_test_encoded)
x_train

,Open,High,Low,Close,Adj Close,Volume,MA,upSwing
507,30.090000,30.180000,29.570000,29.670000,26.592661,35191500,29.949333,1
247,34.540001,34.840000,34.470001,34.580002,29.955334,22867200,32.712000,1
434,33.919998,34.209999,33.650002,33.740002,29.939529,20503300,33.816667,0
1225,42.439999,43.349998,42.259998,43.000000,42.627705,25754600,43.554667,0
775,34.299999,34.540001,33.990002,34.470001,32.067467,26117900,33.852666,0
1088,35.889999,36.360001,35.869999,36.220001,35.330616,17537600,35.947000,1
442,31.420000,31.690001,30.990000,31.330000,27.800985,124645600,33.742334,0
640,34.720001,34.889999,34.680000,34.840000,31.767973,14815500,35.288667,0
467,32.740002,32.889999,32.650002,32.750000,29.061037,16335400,32.516333,1
848,33.639999,33.669998,33.139999,33.220001,31.203733,13924400,32.980667,1


In [20]:
 print(rf.feature_importances_)

[0.12030699 0.13542233 0.14376646 0.12872506 0.15310533 0.12476701
 0.17583652 0.01807031]


In [32]:
print(rf.predict([x.iloc[-1]]))

[0]
